In [1]:
# キャッシュのデータをpandasであれこれする
from datetime import datetime
import pandas as pd
import numpy as np
import pymongo
import json
import os
import sys
import time
import copy
import re
from pprint import pprint
import matplotlib.pyplot as plt
import matplotlib
import sys
import codecs
from janome.tokenizer import Tokenizer

# 形態素解析のセッティング
t = Tokenizer()

# グラフの出力を日本語に指定する。
font = {'family' : 'TakaoGothic'}
matplotlib.rc('font', **font)

# MongoDBの立ち上げ
os.system('sudo service mongod start')

# MongoDBの仮想環境への接続
client = pymongo.MongoClient()
automata_db = client['PretaAutomata']
add_a = automata_db['AddPretaAutomata']

# 取得したい日時のログを収録する。
dt = datetime(2015, 4, 1, 12, 30, 59, 0).timestamp()
d = add_a.find({'meta.timestamp': {'$gte': dt}})

talk_list = list()  # 発言データが色々投げ込まれるゴミ箱

In [5]:
seer_b = r'^\s?(占い|うらない)(CO|ＣＯ)( |　)?.+(●|人狼)'
b_check = re.compile(seer_b)

for v in d:
    fox = [player for player in v['player'] if player['role'] == '妖狐']
    
    # 狐のデータが取得出来ないなら見なかった事にする
    if fox == []:
        continue
    else:
        fox = fox[0]
        
    # COパターンの作成
    seer = 1 if fox['virtual_role'] == '占い師' else 0
    mid = 1 if fox['virtual_role'] == '霊能者' else 0
    
    # 占い○か●かの区別
    if seer == 1:
        result_pattern = '○'
        # 全発言の取得、及びソート
        seer_talk_list = [talk for talk in v['target_log'] if talk['name'] == fox['name']]
        seer_talk_list.reverse()
        
        seer_talk_list.sort(key=lambda x: x['player_talk_id'])
        if len(seer_talk_list) >= 2:
            seer_talk_list = seer_talk_list[0:1]
        
        for seer_talk in seer_talk_list:
            if b_check.match(seer_talk['text']):
                result_pattern = '●'
                break
        
        seer = '1%s' % result_pattern
    
    co_pattern = '%s-%d' % (seer, mid)
    append_data = (co_pattern, v['meta']['victory'])
    # 勝利役職の挿入
    talk_list.append(append_data)  # もはやtalk_listではない

print(talk_list)

[('0-0', 'draw'), ('0-1', 'vill'), ('0-0', 'wolf'), ('0-0', 'wolf'), ('0-0', 'vill'), ('0-0', 'vill'), ('0-0', 'wolf'), ('0-0', 'del'), ('0-0', 'wolf'), ('0-0', 'vill'), ('0-0', 'wolf'), ('0-0', 'vill'), ('0-0', 'wolf'), ('0-0', 'wolf'), ('1○-0', 'wolf'), ('0-0', 'fox'), ('0-0', 'vill'), ('0-0', 'vill'), ('0-0', 'wolf'), ('0-0', 'draw'), ('0-0', 'vill'), ('0-0', 'vill'), ('0-0', 'vill'), ('0-0', 'wolf'), ('0-0', 'vill'), ('0-0', 'wolf'), ('0-0', 'wolf'), ('0-0', 'wolf'), ('0-0', 'wolf'), ('0-0', 'wolf'), ('0-0', 'fox'), ('0-0', 'wolf'), ('0-0', 'vill'), ('0-0', 'draw'), ('0-0', 'vill'), ('0-0', 'wolf'), ('0-0', 'fox'), ('0-0', 'vill'), ('0-0', 'wolf'), ('0-0', 'wolf'), ('1○-0', 'wolf'), ('0-0', 'vill'), ('0-0', 'vill'), ('0-0', 'fox'), ('0-0', 'wolf'), ('0-0', 'vill'), ('0-0', 'fox'), ('1○-0', 'vill'), ('0-0', 'wolf'), ('0-0', 'fox'), ('0-0', 'vill'), ('0-0', 'vill'), ('0-0', 'vill'), ('0-0', 'vill'), ('0-0', 'wolf'), ('0-1', 'vill'), ('0-0', 'del'), ('1●-0', 'vill'), ('0-0', 'fox'), (

In [6]:
df = pd.DataFrame(talk_list, columns=('pattern', 'win_role'))
df

,pattern,win_role
0,0-0,draw
1,0-1,vill
2,0-0,wolf
3,0-0,wolf
4,0-0,vill
5,0-0,vill
6,0-0,wolf
7,0-0,del
8,0-0,wolf
9,0-0,vill


In [7]:
df.groupby('pattern')['win_role'].value_counts()

pattern  win_role
0-0      vill        926
         wolf        509
         fox         232
         draw        124
         del         104
0-1      vill         34
         wolf         26
         fox           7
         draw          6
         del           4
1○-0     wolf         49
         vill         48
         draw          7
         fox           6
         del           3
1●-0     vill         25
         wolf         14
         draw          3
         fox           1
Name: win_role, dtype: int64

In [8]:
df.groupby('pattern')['win_role'].value_counts(normalize=True)

pattern  win_role
0-0      vill        0.488654
         wolf        0.268602
         fox         0.122427
         draw        0.065435
         del         0.054881
0-1      vill        0.441558
         wolf        0.337662
         fox         0.090909
         draw        0.077922
         del         0.051948
1○-0     wolf        0.433628
         vill        0.424779
         draw        0.061947
         fox         0.053097
         del         0.026549
1●-0     vill        0.581395
         wolf        0.325581
         draw        0.069767
         fox         0.023256
Name: win_role, dtype: float64